In [34]:
import pke

In [9]:
import pandas as pd
# pd.set_option('max_colwidth', 800)
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .config("spark.jars.package", "JohnSnowLabs:spark-nlp:1.7.3") \
    .getOrCreate()

dataPath = 'reddit-worldnews.json'
df = spark.read.json(dataPath)
print(df.count())
df.printSchema()

100
root
 |-- data: struct (nullable = true)
 |    |-- approved_at_utc: string (nullable = true)
 |    |-- approved_by: string (nullable = true)
 |    |-- archived: boolean (nullable = true)
 |    |-- author: string (nullable = true)
 |    |-- author_flair_background_color: string (nullable = true)
 |    |-- author_flair_css_class: string (nullable = true)
 |    |-- author_flair_richtext: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- author_flair_template_id: string (nullable = true)
 |    |-- author_flair_text: string (nullable = true)
 |    |-- author_flair_text_color: string (nullable = true)
 |    |-- author_flair_type: string (nullable = true)
 |    |-- author_fullname: string (nullable = true)
 |    |-- author_patreon_flair: boolean (nullable = true)
 |    |-- banned_at_utc: string (nullable = true)
 |    |-- banned_by: string (nullable = true)
 |    |-- can_gild: boolean (nullable = true)
 |    |-- can_mod_post: boolean (nullable = true)


In [28]:
dfTitle = df.select("data.title")
# dfTitle.limit(5).toPandas()
titles = dfTitle.select("*").toPandas()['title']

In [31]:
raw_text = ' '.join(titles)

extractor = pke.unsupervised.TopicRank()
extractor.load_document(input=raw_text, language='en')
extractor.candidate_selection()
extractor.candidate_weighting()
keyphrases = extractor.get_n_best(n=10)

keyphrases

[('trump organization', 0.013136112408202718),
 ('new hypersonic missile', 0.010843860682135789),
 ('russia', 0.010546846209338444),
 ('saudi arabia', 0.010395904385254628),
 ('climate change', 0.010255616800911648),
 ('secret', 0.00872196161104388),
 ('india president vladimir putin', 0.008392157879422673),
 ('years', 0.0077976570008257695),
 ('strategic weapons', 0.006963862606277846),
 ('social media fears duterte', 0.006703610587080468)]

In [36]:
from nltk.corpus import stopwords
# docs: https://boudinfl.github.io/pke/build/html/unsupervised.html#yake

# 1. create a YAKE extractor.
extractor = pke.unsupervised.YAKE()

# 2. load the content of the document.
extractor.load_document(input=raw_text,
                        language='en',
                        normalization=None)


# 3. select {1-3}-grams not containing punctuation marks and not
#    beginning/ending with a stopword as candidates.
stoplist = stopwords.words('english')
extractor.candidate_selection(n=3, stoplist=stoplist)

# 4. weight the candidates using YAKE weighting scheme, a window (in
#    words) for computing left/right contexts can be specified.
window = 2
use_stems = True # use stems instead of words for weighting
extractor.candidate_weighting(window=window,
                              stoplist=stoplist,
                              use_stems=use_stems)

# 5. get the 10-highest scored candidates as keyphrases.
#    redundant keyphrases are removed from the output using levenshtein
#    distance and a threshold.
threshold = 0.8
keyphrases = extractor.get_n_best(n=10, threshold=threshold)
keyphrases

[('president vladimir putin', 0.0058101553951246795),
 ('targeting democratic institutions', 0.005995838504790576),
 ('saudi arabia', 0.0060746287754789375),
 ('discovered hacking targeting', 0.006168005441389686),
 ('hacking targeting democratic', 0.006168005441389686),
 ('saudi arabia nuclear', 0.006347582122216154),
 ('microsoft corp said', 0.006793816760011038),
 ('technology to saudi', 0.00840360825517643),
 ('organizations in europe', 0.0089866034379013),
 ('vladimir putin', 0.009253261160451914)]